# Benchmark
This notebook shows the benchmark results of all covered ops.

## Setup

In [1]:
from IPython.display import clear_output

def build_package(name, url, commit=None, deps=True):
    import importlib
    import os, sys
    if importlib.util.find_spec(name) is None:
        os.system(f"git clone {url} {name} || true")
        if commit is not None:
            os.system(f"cd {name}; git checkout {commit}")
        os.system(f"cd {name}; git submodule update --init --recursive")
        no_deps = ""
        if deps:
            os.system(f"cd {name}; pip3 install -r requirements.txt || true")
        else:
            no_deps = "--no-deps"
        clear_output()
        os.system(f'cd {name}; pip3 install -e ".[dev]" {no_deps}')

build_package("transformers", "https://github.com/huggingface/transformers.git", deps=False)
build_package("xformers", "https://github.com/facebookresearch/xformers.git")
build_package("epoi", "https://github.com/comaniac/epoi.git")

## Benchmark

In [2]:
from IPython.display import display, Javascript

disable_js = """
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}
"""

def load_ipython_extension():
    display(Javascript(disable_js))
    print ("autoscrolling long output is disabled")
    
load_ipython_extension()

<IPython.core.display.Javascript object>

autoscrolling long output is disabled


In [3]:
!python3 -m epoi.benchmark

===== Environment =====

GPU: Tesla V100-SXM2-16GB

PyTorch Configuration
   Config         Value
-------------  ------------
   Version     1.13.0+cu117
Built w. CUDA      11.7


Other Libraries Configuration
  Package       Version                   Commit SHA
------------  -----------  ----------------------------------------
    epoi        0.1.dev    cd79b19e14a81672f6a62bce7054827352f5eade
transformers  4.25.0.dev0  2bdd9fa28411a2822cd1395ed78abeef4a69ec6f
  xformers    0.0.14.dev   ba93c5012d00bd1b010514a7bc9bd938c1ad6149
  megatron        N/A      0bb597b42c53355a567aba2a1357cc34b9d99ddd
   triton        2.0.0                       N/A
    apex          0.1                        N/A
===== Environment =====

[2022-11-10 00:32:33] INFO main: Selected bias_gelu
[2022-11-10 00:32:33] INFO main: Selected dropout_add_ln
[2022-11-10 00:32:33] INFO main: Selected bert_attention
[2022-11-10 00:32:33] INFO main: Selected gpt_attention
[2022-11-10 00:32:33] INFO main: Selected qkv_self_a

[2022-11-10 00:52:54] INFO bencher: Correctness checking for xFormers FlashAttn (native w/o weight scaling) is passed
[2022-11-10 00:52:54] INFO layer_ops: Skip correctness checking for CUTLASS and Triton due to not support weight scaling
[-------- CrossAttention: T5 (Attn) and FlashAttention (FA) --------]
                               |  HF (Attn)  |  xFormers cutlass (FA)
1 threads: ----------------------------------------------------------
      (4, 1024, 512, 64, 16)   |     10.1    |            5.4        
      (4, 1024, 1024, 64, 16)  |     10.6    |            6.0        
      (4, 2048, 1024, 64, 16)  |     38.8    |           20.2        

Times are in milliseconds (ms).

         Shape            HF (Attn)    xFormers cutlass (FA)
-----------------------  -----------  -----------------------
(4, 1024, 512, 64, 16)      1181              292.25
(4, 1024, 1024, 64, 16)     1186              296.25
(4, 2048, 1024, 64, 16)     4674              1104.5

Memory is in MBs and exc